# Data Exploration and Technical Indicators

This notebook demonstrates how to load stock data and calculate technical indicators.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data import DataLoader, TechnicalIndicators
from config import DATA_CONFIG

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (15, 8)

## 1. Load Stock Data

In [ ]:
# Initialize data loader
ticker = 'AAPL'
loader = DataLoader(ticker, period='2y')

# Fetch data
data = loader.fetch_data()
print(f"Loaded {len(data)} data points")
data.head()

## 2. Basic Statistics

In [ ]:
# Display statistics
data.describe()

In [ ]:
# Plot closing price
plt.figure(figsize=(15, 6))
plt.plot(data.index, data['Close'], label='Close Price')
plt.title(f'{ticker} Stock Price')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.show()

## 3. Add Technical Indicators

In [ ]:
# Add all technical indicators
data_with_indicators = TechnicalIndicators.add_all_indicators(data)
print(f"Added {len(data_with_indicators.columns) - 5} indicators")
data_with_indicators.columns.tolist()

## 4. Visualize Technical Indicators

In [ ]:
# Plot price with moving averages
fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

# Price and MAs
axes[0].plot(data_with_indicators.index, data_with_indicators['Close'], label='Close', linewidth=2)
axes[0].plot(data_with_indicators.index, data_with_indicators['SMA_20'], label='SMA 20', alpha=0.7)
axes[0].plot(data_with_indicators.index, data_with_indicators['SMA_50'], label='SMA 50', alpha=0.7)
axes[0].set_ylabel('Price ($)')
axes[0].set_title('Price with Moving Averages')
axes[0].legend()
axes[0].grid(True)

# RSI
axes[1].plot(data_with_indicators.index, data_with_indicators['RSI'], label='RSI', color='purple')
axes[1].axhline(y=70, color='r', linestyle='--', alpha=0.5, label='Overbought')
axes[1].axhline(y=30, color='g', linestyle='--', alpha=0.5, label='Oversold')
axes[1].set_ylabel('RSI')
axes[1].set_title('Relative Strength Index')
axes[1].legend()
axes[1].grid(True)

# MACD
axes[2].plot(data_with_indicators.index, data_with_indicators['MACD'], label='MACD', color='blue')
axes[2].plot(data_with_indicators.index, data_with_indicators['MACD_signal'], label='Signal', color='red')
axes[2].bar(data_with_indicators.index, data_with_indicators['MACD_diff'], label='Histogram', alpha=0.3)
axes[2].set_ylabel('MACD')
axes[2].set_xlabel('Date')
axes[2].set_title('MACD')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

## 5. Current Trading Signals

In [ ]:
# Get current signals
signals = TechnicalIndicators.get_current_signals(data_with_indicators)

print("Current Trading Signals:")
print("=" * 50)
for indicator, signal in signals.items():
    print(f"{indicator:15s}: {signal}")

## 6. Correlation Analysis

In [ ]:
# Select key indicators for correlation
key_indicators = ['Close', 'SMA_20', 'SMA_50', 'RSI', 'MACD', 'Volume']
correlation_data = data_with_indicators[key_indicators].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Key Indicators')
plt.show()